In [119]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [120]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from bs4 import BeautifulSoup #For web Scrabing
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [121]:
# assigning web page to wiki_url
wiki_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [122]:
# requesting from web page wiki_url
response=requests.get(wiki_url)

In [123]:
#  returns text of a html file
soup=BeautifulSoup(response.text,'html.parser')

In [124]:
# finding specific table with the class of wikitable sortable
torontoadress=soup.find_all('table',class_='wikitable sortable')[0]

In [125]:
# a new table to fill up with the data
tablenew=[]

In [126]:
# looping inside each row (tr) and its column cell (td)  of the table within the HTML file & then assigning each data under 3 columns 
for row in torontoadress.find_all('tr'):
    cols = row.find_all('td')
    if len(cols) == 3:
        tablenew.append((cols[0].text.strip(), cols[1].text.strip(), cols[2].text.strip()))

In [127]:
# List to Pandas dataframe 

df = pd.DataFrame(tablenew)

df.columns = ['Postal Code', 'Borough', 'Neighbourhood']
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [128]:
# dropping "Not assigned" cells in " Borough" column and resetting index  

df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df = df.reset_index(drop=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [129]:
# Shape fo the frame
df.shape

(103, 3)

## Part2

In [130]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: click in c:\users\aydinakg\anaconda3\lib\site-packages (from geocoder) (7.1.2)



In [131]:
pip install --upgrade geocoder

Requirement already up-to-date: geocoder in c:\users\aydinakg\anaconda3\lib\site-packages (1.38.1)
Note: you may need to restart the kernel to use updated packages.


In [132]:
import geocoder

In [133]:
# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates

g = geocoder.google('Turkey')
lat_lng_coords = g.latlng

print(lat_lng_coords)

None


### For some reason geocoder is not working, therefore I uploaded csv file and merge 'df' & 'lat_long_data' dataframes while matching the postal codes


In [134]:
lat_long_data=pd.read_csv('Geospatial_Coordinates.csv')

In [135]:
merged_df = df.merge(lat_long_data, how = 'inner', on = ['Postal Code'])

In [140]:
merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [141]:
# Just Trial 
toronto_merged_df=merged_df[merged_df['Borough'].str.contains('Toronto',case=False)].reset_index(drop=True)
toronto_merged_df.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [142]:
address = 'Toronto'

geolocator = Nominatim(user_agent="CA_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [143]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(toronto_merged_df['Latitude'], toronto_merged_df['Longitude'], toronto_merged_df['Borough'], toronto_merged_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)


map_toronto

In [144]:
def func(row):
    if row['Borough'] == 'West Toronto':
        return 'West'
    elif row['Borough'] == 'East Toronto':
        return 'East' 
    elif row['Borough'] == 'Central Toronto':
        return 'Central' 
    elif row['Borough'] == 'Downtown Toronto':
        return 'Downtown' 
    else:
        return 'other'

toronto_merged_df['Label'] = toronto_merged_df.apply(func, axis=1)

toronto_merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Label
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Downtown
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Downtown
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Downtown
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,Downtown
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,East


In [166]:
# Another method - just trial

def func(row):
    if 'West' in row['Borough']:
        return 1
    elif 'East' in row['Borough']:
        return 2
    elif 'Central' in row['Borough']:
        return 3 
    elif 'Downtown' in row['Borough']:
        return 4 
    else:
        return 'other'

toronto_merged_df['Label'] = toronto_merged_df.apply(func, axis=1)
toronto_merged_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Label
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,4
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,4
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,4
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2


In [167]:
toronto_grouped = toronto_merged_df.groupby('Label').mean().reset_index()

toronto_grouped

,Label,Latitude,Longitude
0,1,43.652653,-79.449290
1,2,43.669436,-79.324654
2,3,43.701980,-79.398954
3,4,43.654597,-79.383972
